In [120]:
# this implementation gets "N_total" and "N_spin" vectors from the 
# atoms in the cage of the diels alder rxn

def extract_charge_energies(num, filename = "reactC_endo10MVc.sum"):
    with open(filename) as myFile:        
        control = 0
        iter = 0 
        
        ret_list = []
        ret_list1 = []
        ret_list2 = []
        
        for line_num, line in enumerate(myFile.readlines()):
            try:
                if (line.split()[0] == "Molecular" and line.split()[1] == "energy"):
                    ret_list.append(float(line.split()[-1]))
                    
                if (line.split()[0] == "Some" and line.split()[1] == "Atomic"
                   and line.split()[2] == "Properties:"):
                    control = 1

                if (num[iter] == line.split()[0] and control == 1):
                    
                    ret_list1.append(float(line.split()[1]))
                    ret_list2.append(float(line.split()[2]))                    
                    iter += 1
                    
            except:
                pass
            
    ret_list1 = ret_list +  ret_list1 + ret_list2
    return ret_list1

# this implementation gets "N_total" and "N_spin" vectors from the 
# atoms in the cage of the diels alder rxn

def extract_spin(num, filename = "reactC_endo10MVc.sum"):
    with open(filename) as myFile:        
        control = 0
        ret_list1 = []
        ret_list2 = []

        iter = 0 
        for line_num, line in enumerate(myFile.readlines()):
            try:
                
                if (line.split()[0] == "Atomic" and line.split()[1] == "Electronic"
                   and line.split()[2] == "Spin"):
                    control = 1

                if (num[iter] == line.split()[0] and control == 1):
                    
                    ret_list1.append(float(line.split()[3]))
                    ret_list2.append(float(line.split()[4]))     
                    iter += 1
    
                    
            except:
                pass
            
    ret_list1 = ret_list1 + ret_list2
    return ret_list1

extract_charge_energies(num = ["c1","c3","c4","c5","c6","c15"])


here


[-364.9144228,
 0.2395089916,
 -0.0208678928,
 -0.5790717917,
 -0.1707778405,
 -0.2298008991,
 0.8493317765,
 0.00020909625741,
 0.00047979863849,
 4.4382217793e-05,
 -0.00030788005912,
 -0.0001031478033,
 7.5199839332e-05]

In [114]:
# note, might have to modulate the string traversing based on the number of atoms in the file 
# bond pulling stil isn't achieved

def extract_basics(num, filename = "reactC_endo10MVc.sum"):
    
    lookup_dictionary  = [
    "Number of electrons",
    "Nuclear Charges and Cartesian Coordinates",
    "n_atoms",
    "L(A)",
    "Molecular energy E(Mol) from the wfn file:",
    "1st Largest",
    "Atomic Electronic Spin Populations",
    "Atomic Dipole Moments:",
    ]
    
    
    
    control_1 = 0
    control_2 = 0

    ret_list = []
    iter = 0

    with open(filename) as myFile:
        for line_num, line in enumerate(myFile.readlines()):
            try:
                
                
                ########
                if(iter > 2 * len(num) - 1):
                    break
                    
                    
                if (line.split()[1] == "Atomic" and 
                    line.split()[0] == "Some"):
                    control_1 = 1
                 
                   
                if (line.split()[1] == "Charges" and 
                    line.split()[0] == "Nuclear"):
                    control_2 = 1
                
                #grabs charge, lagrangian, kinetic energy of ea. atom
                if(int(line.split().count(num[iter%len(num)])) > 0 and control_1 > 0):
                    ret_list.append(line.split()[1:4])
                    iter += 1
                    
                #grabs position of six atoms
                if(int(line.split().count(num[iter%len(num)])) > 0 and control_2 > 0):
                    ret_list.append(line.split()[2:5])
                    iter += 1
                 
            except: 
                pass
            
        return ret_list
    
extract_basics(num = ["c1","c3","c4","c5","c6","c15"])

[['-4.2704040000E-02', '3.3422718000E-01', '-4.0258855000E-01'],
 ['-9.7538808000E-01', '1.1435038700E+00', '3.7418277800E+00'],
 ['1.2963342500E+00', '2.2832510000E-02', '3.8132104600E+00'],
 ['2.1080071000E+00', '-6.1809590000E-01', '1.1772669300E+00'],
 ['3.5369486100E+00', '5.6687716900E+00', '-1.7657576000E-01'],
 ['1.7630271000E-01', '6.9240178200E+00', '2.6252554600E+00'],
 ['2.3950899160E-01', '2.0909625741E-04', '3.9505328813E+01'],
 ['-2.0867892800E-02', '4.7979863849E-04', '3.7892029257E+01'],
 ['-5.7907179170E-01', '4.4382217793E-05', '3.9951978392E+01'],
 ['-1.7077784050E-01', '-3.0788005912E-04', '4.0021407948E+01'],
 ['-2.2980089910E-01', '-1.0314780330E-04', '3.9414668027E+01'],
 ['8.4933177650E-01', '7.5199839332E-05', '3.7925989048E+01']]

In [65]:
#  CPs have different lengths, some dont have gbl
#  types of critical points - might have to one-hot encode
#  NACP = Nuclear Attractor Critical Point                                      
#  NNACP = Non-Nuclear Attractor Critical Point    
#  BCP = Bond Critical Point                                                    
#  RCP = Ring Critical Point                                                    
#  CCP = Cage Critical Point

# Todo - make method to pull out a single critical points
# each critical point has:
#---------------------------------------------
#   Rho = Electron Density   
#   DelSqRho = Laplacian of Rho = Trace of Hessian of Rho                        
#   DelSqV = Laplacian of V                                                      
#   ask matthew about all the other types of laplacians
#   HessRho_EigVals = Eigenvalues of the Hessian of Rho, Ascending Order         
#   Bond Ellipticity = (HessRho_EigVal(1) / HessRho_EigVal(2)) - 1               
#   G = Lagrangian Form of Kinetic Energy Density                                
#   K = Hamiltonian Form of Kinetic Energy Density                               
#   L = K - G = Lagrangian Density = (-1/4)DelSqRho        

#   Stress_EigVals = Eigenvalues of Stress Tensor, Ascending Order               
#   V = Virial Field = Potential Energy Density = Trace of Stress Tensor         
#  -DivStress = Ehrenfest Force Density = Minus Divergence of Stress Tensor     


# num is an array of the critical nulcear atoms
def extract_nuc_crit(num,filename = "reactC_endo10MVc.sum"):

    lookup_nuclear = [
        "Rho",
        "GradRho",
        "HessRho_EigVals",
        "DelSqRho",
        "V",
        "G",
        "K",
        "L",
        "Vnuc",
        "DelSqV",
        "Stress_EigVals",
        "-DivStress",
        "ESP",
        "ESPe",
        "ESPn",
    ]
     

    num = sorted(num)
    ret_list = {}
    list_dicts = []
    iter = 0
    iter_lookup = 0
    control = 0 
    
    with open(filename) as myFile:        
        for line_num, line in enumerate(myFile.readlines()):
            
            try: 
                if(control == 1):

                    if(iter_lookup == len(lookup_nuclear) or line.split() == []):
                        iter_lookup = 0
                        control = 0
                        iter += 1
                        list_dicts.append(ret_list)
                        ret_list = {}

                    if (line.split()[0] == "Type"):
                        ret_list.append(line.split()[3])
                        
                    if(lookup_nuclear[iter_lookup] == line.split()[0]):
                        if(line.split()[0] == "Stress_EigVals"):
                            ret_list["Stress_EigVals"] = line.split()[2:5]
                        elif(line.split()[0] == "GradRho"):
                            ret_list["GradRho"] = line.split()[2:5]
                        elif(line.split()[0] == "HessRho_EigVals"):
                            ret_list["HessRho_EigVals"] = line.split()[2:5]
                              
                        else:
                            ret_list[lookup_nuclear[iter_lookup]] = line.split()[2]
    
                        iter_lookup += 1
                    
                if (int(line.split()[1]) == num[iter] and 
                    line.split()[0] == "CP#"):
                    control = 1
                    
            except: 
                pass
        return list_dicts
        

extract_nuc_crit(num = [3,4,5,6,15])


[{'Rho': '1.2219248186E+02',
  'GradRho': ['-4.3245445003E-09', '1.5603660586E-09', '-1.7091323912E-09'],
  'HessRho_EigVals': ['-3.2301760542E+05',
   '-3.2301548229E+05',
   '-3.2301456790E+05'],
  'DelSqRho': '-9.6904765561E+05',
  'V': '-2.4227927950E+05',
  'G': '8.6828001323E+00',
  'K': '2.4227059670E+05',
  'L': '2.4226191390E+05',
  'Vnuc': '4.8811587530E+06',
  'DelSqV': '2.3768528932E+10',
  'Stress_EigVals': ['-8.0760229117E+04',
   '-8.0759914070E+04',
   '-8.0759136315E+04'],
  '-DivStress': '8.4482900097E+03',
  'ESP': '-1.4877659930E+01',
  'ESPe': '-2.7832952538E+01',
  'ESPn': '1.2955292608E+01'},
 {'Rho': '1.2314342426E+02',
  'GradRho': ['-4.9098503041E-12', '-3.1397107136E-13', '-6.8394845343E-11'],
  'HessRho_EigVals': ['-3.2539932130E+05',
   '-3.2539128140E+05',
   '-3.2538878489E+05'],
  'DelSqRho': '-9.7617938759E+05',
  'V': '-2.4407093033E+05',
  'G': '1.3041713957E+01',
  'K': '2.4405788861E+05',
  'L': '2.4404484690E+05',
  'Vnuc': '6.7132109477E+05',
  'D

In [66]:
# For all nuclear critical points, there is 
# i think this all are in the other part 
# (1) Bader charge, - the other
# (2) energy, - in the other 
# (3) electrostatic potential,  
# (4) nuclear electrostatic potential,
#---------------------------------------------
#  Vnuc = Electrostatic Potential from Nuclei                                   
#   ESP = Total Electrostatic Potential                                          
#  ESPe = Electrostatic Potential from Electrons                                
#  ESPn = Electrostatic Potential from Nuclei  

# (5) multipoles, 
# (6) force tensors, 

# (7) spin populations - Done 
#---------------------------------------------
#Atomic Electronic Spin Populations:
#...


def extract_other_crit(num, filename = "reactC_endo10MVc.sum"):
    lookup_other = [
        "Rho",
        "GradRho",
        "HessRho_EigVals",
        "DelSqRho",
        "Bond",
        "V",
        "G",
        "K",
        "L",
        "Vnuc",
        "DelSqV",
        "Stress_EigVals",
        "-DivStress",
        "ESP",
        "ESPe",
        "ESPn",
        "GBL"
    ]
    
    
    num = sorted(num)
    iter = 0
    iter_lookup = 0
    control = 0 
    ret_list = {}
    list_dicts = []
    
    with open(filename) as myFile:        
        for line_num, line in enumerate(myFile.readlines()):
            
            try: 
                if(control == 1):

                    if(iter_lookup == len(lookup_other) or line.split() == []):
                        iter_lookup = 0
                        control = 0
                        iter += 1
                        list_dicts.append(ret_list)
                        ret_list = {}

                    if (line.split()[0] == "Type"):
                        ret_list.append(line.split()[3])

                    if(lookup_other[iter_lookup] == line.split()[0]):

                        if(line.split()[0] == "Stress_EigVals"):
                            ret_list["Stress_EigVals"] = line.split()[2:5]
                        elif(line.split()[0] == "GradRho"):
                            ret_list["GradRho"] = line.split()[2:5]
                        elif(line.split()[0] == "HessRho_EigVals"):
                            ret_list["HessRho_EigVals"] = line.split()[2:5]
                        else:
                            ret_list[lookup_other[iter_lookup]] = line.split()[2]
    
                        iter_lookup += 1
                    
                if (int(line.split()[1]) == num[iter] and 
                    line.split()[0] == "CP#"):
                    control = 1
                    
            except: 
                pass
        return list_dicts
    
    
extract_other_crit(num = [31,22,23,24,25,32,41])


[{'Rho': '1.9162213298E-01',
  'GradRho': ['7.6327832943E-17', '-1.8735013541E-16', '-4.3368086899E-17'],
  'HessRho_EigVals': ['-3.5461870118E-01',
   '-3.3747657073E-01',
   '1.1185208657E+00'],
  'DelSqRho': '4.2642559382E-01',
  'Bond': '=',
  'V': '-4.4331847892E-01',
  'G': '2.7496243869E-01',
  'K': '1.6835604023E-01',
  'L': '-1.0660639845E-01',
  'Vnuc': '1.8803557232E+01',
  'DelSqV': '-1.3241607953E+00',
  'Stress_EigVals': ['-1.6349065603E-01',
   '-1.4656326556E-01',
   '-1.3326455733E-01'],
  '-DivStress': '-3.4248172630E-03',
  'ESP': '-2.5628016599E-01',
  'ESPe': '-1.9059837398E+01',
  'ESPn': '1.8803557232E+01'},
 {'Rho': '3.4555051209E-01',
  'GradRho': ['1.7069679004E-15', '-6.3837823916E-16', '-5.8286708793E-16'],
  'HessRho_EigVals': ['-7.0722165815E-01',
   '-5.8804311525E-01',
   '3.2966152300E-01'],
  'DelSqRho': '-9.6560325040E-01',
  'Bond': '=',
  'V': '-5.5614147466E-01',
  'G': '1.5737033103E-01',
  'K': '3.9877114363E-01',
  'L': '2.4140081260E-01',
  'Vn

In [115]:
extract_charge_energies(num = ["c1","c3","c4","c5","c6","c15"])


[0.2395089916,
 -0.0208678928,
 -0.5790717917,
 -0.1707778405,
 -0.2298008991,
 0.8493317765,
 0.00020909625741,
 0.00047979863849,
 4.4382217793e-05,
 -0.00030788005912,
 -0.0001031478033,
 7.5199839332e-05]

In [68]:
extract_spin(num = ["c1","c3","c4","c5","c6","c15"])

[5.7604910084,
 6.0208678928,
 6.5790717917,
 6.1707778405,
 6.2298008991,
 5.1506682235,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

TypeError: list indices must be integers or slices, not str